In [ ]:
from tensorflow.keras import layers, models

def build_cnn_model(input_shape, output_shape):
    
    # Building CNN: encoder (2D Convolutions to process 2D images)
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    
    # Add a flatten and dense layers to create bottleneck (latent) representation
    model.add(layers.Flatten())
    model.add(layers.Dense(4 * 4 * 4 * 64, activation='relu'))

    # Reshape to 3D for the decoder
    model.add(layers.Reshape((4, 4, 4, 64)))
    
    # Decoder (3D Convolutions to generate voxel grid)
    model.add(layers.Conv3DTranspose(64, (3, 3, 3), strides=(2, 2, 2), padding='same', activation='relu'))
    model.add(layers.Conv3DTranspose(32, (3, 3, 3), strides=(2, 2, 2), padding='same', activation='relu'))
    model.add(layers.Conv3DTranspose(16, (3, 3, 3), strides=(2, 2, 2), padding='same', activation='relu'))
    model.add(layers.Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same'))  # Sigmoid for binary voxel grid

    return model

# Define the input and output shapes, then instantiate and compile the model
input_shape = (128, 128, 1)
output_shape = (32, 32, 32, 1)
model = build_cnn_model(input_shape, output_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Load your 2D images and 3D voxel grids from the saved .npy files
images = np.load('02691156_images.npy')
voxel_grids = np.load('02691156_voxels.npy')

print(f"Shape of the images data: {images.shape}")
print(f"Shape of the voxel grids data: {voxel_grids.shape}")

# Training the model
model_history = model.fit(images, voxel_grids, epochs=10, batch_size=16, validation_split=0.1)

In [ ]:
model.summary()

In [ ]:
print(model.metrics_names)
model_history.history

In [ ]:
# Save the paths of the testing image into a list
DATA_PATH = "."
png_file = get_png_files(DATA_PATH, recursive=False)

# Convert a single testing image into NPY
save_npy_array_from_png(png_files[1], (128, 128), npy_filename='test_image')

In [ ]:
# Load the image and reshape adding a batch dimension for prediction
image_npy = np.load('02691156_images.npy')[1114]
image_npy = image_npy.reshape(1, 128, 128, 1)

# Make a Prediction and remove batch channel from prediction
predicted_voxel = model.predict(image_npy)
predicted_voxel = predicted_voxel[0, :, :, :, 0]

# Convert the voxel grid to a binary format (can modify threshold)
predicted_voxel_binary = (predicted_voxel > 0.5).astype(np.uint8)

In [ ]:
def save_voxel_grid_to_obj(voxel_grid, obj_filename):
    # Vertices and faces for an individual cube (each voxel)
    vertex_offsets = [
        (0, 0, 0), (1, 0, 0), (1, 1, 0), (0, 1, 0),
        (0, 0, 1), (1, 0, 1), (1, 1, 1), (0, 1, 1)
    ]
    
    face_offsets = [
        (0, 1, 2, 3), # Bottom
        (4, 5, 6, 7), # Top
        (0, 1, 5, 4), # Front
        (2, 3, 7, 6), # Back
        (1, 2, 6, 5), # Right
        (3, 0, 4, 7)  # Left
    ]

    vertices = []
    faces = []
    vertex_index = 1

    # Iterate through the 3D array and process each voxel
    for x in range(voxel_grid.shape[0]):
        for y in range(voxel_grid.shape[1]):
            for z in range(voxel_grid.shape[2]):
                if voxel_grid[x, y, z] == 1:  # Only process occupied voxels (value 1)
                    # Add vertices for this voxel's cube
                    for dx, dy, dz in vertex_offsets:
                        vertices.append((x + dx, y + dy, z + dz))

                    # Add faces using the latest 8 vertices
                    for face in face_offsets:
                        faces.append((
                            vertex_index + face[0],
                            vertex_index + face[1],
                            vertex_index + face[2],
                            vertex_index + face[3]
                        ))

                    # Update the vertex index
                    vertex_index += 8

    # Write vertices and faces to an OBJ file
    with open(obj_filename, 'w') as obj_file:
        for vertex in vertices:
            obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        for face in faces:
            obj_file.write(f"f {face[0]} {face[1]} {face[2]} {face[3]}\n")


# Save the voxel grid to an OBJ file
save_voxel_grid_to_obj(predicted_voxel_binary, 'test_voxel.obj')